In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Análisis exploratorio + Pre-procesamiento - Dataset MSKCC

In [ ]:
os.makedirs('./ISIC_MSKCC', exist_ok=True)

In [ ]:
#Leemos el dataset 
data = pd.read_csv('./ISIC_MSKCC/mskcc-2020-metadata.csv')
data

In [ ]:
# Eliminamos columnas no necesarias para el análisis
columns_to_drop = ['attribution', 'copyright_license', 'clin_size_long_diam_mm', 'concomitant_biopsy', 'diagnosis_confirm_type', 'image_type', 'mel_class', 'mel_mitotic_index', 'mel_thick_mm', 'mel_type', 'mel_ulcer', 'melanocytic']
data.drop(columns=columns_to_drop, inplace=True)
data = data.dropna()
data

In [ ]:
for column in data.columns:
    unique_values = data[column].unique()
    print(f"Atributo: {column}, Valores únicos: {unique_values}")

In [ ]:
# Comprobación de valores null por columna
valores_nulos_por_columna = data.isnull().sum()
print(valores_nulos_por_columna)

In [ ]:
# Resumen de información del DataFrame
info_df = data.info()
print(info_df)

In [ ]:
# Eliminamos las muestras de diagnosis que no es posible agrupar por recomendación del dermatólogo
data = data[data['diagnosis'] != "atypical melanocytic proliferation"]
data = data[data['diagnosis'] != "AIMP"]
data = data[data['diagnosis'] != "neurofibroma"]

# Mapeamos las localizaciones para que se adecuen al resto de datasets por recomendación del dermatólogo
category_mapping = {
    'lentigo simplex': 'solar lentigo',
    'lentigo NOS': 'solar lentigo',
}

data['diagnosis'] = data['diagnosis'].replace(category_mapping)

data = data[data['diagnosis'] != "verruca"]
data = data[data['diagnosis'] != "acrochordon"]
data = data[data['diagnosis'] != "scar"]
data = data[data['diagnosis'] != "cafe-au-lait macule"]
data = data[data['diagnosis'] != "other"]
data = data[data['diagnosis'] != "angiokeratoma"]

data = data.dropna(subset=['diagnosis'])

In [ ]:
# Realizamos el mapeo final en base a las recomendaciones del dermatólogo
diagnosis_mapping = {
'nevus': 'nevus',                      
'solar lentigo': 'solar lentigo',        
'melanoma': 'melanoma',                   
'lichenoid keratosis': 'keratosis',        
'seborrheic keratosis': 'keratosis',
'actinic keratosis': 'keratosis',
'vascular lesion': 'vascular lesion',             
'dermatofibroma': 'dermatofibroma',
'basal cell carcinoma':'carcinoma',
'squamous cell carcinoma':'carcinoma'
}  

data['diagnosis'] = data['diagnosis'].replace(diagnosis_mapping)

In [ ]:
# Comprobamos los nuevos valores de la columna diagnosis
unique_values = data['diagnosis'].unique()
unique_values

In [ ]:
count_data = data['diagnosis'].value_counts().reset_index()
count_data.columns = ['diagnosis', 'Count']
ax = sns.barplot(data=count_data, x='Count', y='diagnosis', hue='diagnosis', dodge=False, legend=False)
plt.xlabel('Número de pacientes')
plt.ylabel('Diagnóstico')
plt.title('Pacientes según el tipo de diagnóstico')
plt.show()

In [ ]:
# Contamos los valores de 'age_approx', incluyendo NaN
pvc = data['age_approx'].value_counts(dropna=False)
edad_data = data.dropna(subset=['age_approx'])

In [ ]:
# Imprimimos la cantidad de pacientes sin 'age_approx' informada y su porcentaje
age_min = data["age_approx"].min()
age_max = data["age_approx"].max()
age_mean = data["age_approx"].mean()
age_std = data["age_approx"].std()

print("Estadísticas de edad de los pacientes:")
print(f"Mínimo: {age_min}")
print(f"Máximo: {age_max}")
print(f"Media: {age_mean}")
print(f"Desviación estándar: {age_std}")

sns.set(style="whitegrid", palette="pastel")
sns.histplot(data=data, x='age_approx', bins=30, kde=True, color='#6AB7F9')
plt.xlabel('Edad Aproximada')
plt.ylabel('Número de Pacientes')
plt.title('Distribución de Edades de los Pacientes')
plt.show()

In [ ]:
# Filtramos datos por género
female_px = data[data.sex == 'female']
male_px = data[data.sex == 'male']

# Configuramos estilo y paleta de colores de Seaborn
sns.set(style="whitegrid", palette="pastel")

# Creamos histogramas separados por género
sns.histplot(data=female_px, x='age_approx', color='#FABCB7', label='mujer', kde=True)
sns.histplot(data=male_px, x='age_approx', color='#B7D5FA', label='hombre', kde=True)

plt.xlabel('Edad Aproximada')
plt.ylabel('Número de Pacientes')
plt.title('Distribución de Edades de los Pacientes por Género')
plt.legend()
plt.show()

In [ ]:
count_data = data['benign_malignant'].value_counts().reset_index()
count_data.columns = ['Diagnosis', 'Count']

# Count de casos benignos y malignos
num_benign = count_data[count_data['Diagnosis'] == 'benign']['Count'].values[0]
num_malignant = count_data[count_data['Diagnosis'] == 'malignant']['Count'].values[0]

ax = sns.barplot(data=count_data, x='Diagnosis', y='Count', hue='Diagnosis', dodge=False, legend=False)
plt.xlabel('Tipo')
plt.ylabel('Count')
plt.title('Ratio de benigno versus maligno\nBenignos: {} | Malignos: {}'.format(num_benign, num_malignant))
plt.show()

In [ ]:
#Eliminamos las muestras de tipo indeterminate
data = data[data['benign_malignant'] != "indeterminate"]

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
ax = sns.countplot(x='diagnosis', hue='anatom_site_general', data=data, palette='pastel')
for i,v in enumerate(data.anatom_site_general.unique()):
  try:
    ax.bar_label(ax.containers[i])
  except:
    continue

# Ajuste de las etiquetas del eje x para evitar solapamientos
plt.xticks(rotation=45, ha='right')

# Muestra el gráfico
plt.tight_layout()
plt.show()

In [ ]:
# Categorizamos las variables para poder utilizarlas en el modelo 
sex_mapping = {'male': 0, 'female': 1}

diagnosis_mapping = {'nevus': 0,                      
'basal cell carcinoma': 1,        
'melanoma': 2,                   
'squamous cell carcinoma': 3,    
'seborrheic keratosis': 4,        
'actinic keratosis': 5,           
'vascular lesion': 6,             
'dermatofibroma': 7,              
'solar lentigo': 8,               
'lichenoid keratosis': 9 }  

benign_malignant_mapping = {'benign': 0, 'malignant': 1}

anatom_site_general_mapping = {
'head/neck': 0,          
'posterior torso': 1,    
'lower extremity': 2,  
'anterior torso': 3,   
'upper extremity': 4,    
'palms/soles': 5,         
'lateral torso': 6,        
'oral/genital': 7 }  

data['sex'] = data['sex'].replace(sex_mapping)

data.rename(columns={'age_approx': 'age'}, inplace=True)
data.rename(columns={'isic_id': 'id'}, inplace=True)

data['age'] = data['age'].astype(int)
data['sex'] = data['sex'].astype(int)

In [ ]:
data

In [ ]:
# Ahora obtenemos una lista de los nombres de archivos correspondientes a las imágenes dermoscópicas
archivos_dermoscopic = data['id'].tolist()
archivos_dermoscopic = [id + ".JPG" for id in archivos_dermoscopic]

# Ruta de la carpeta que contiene todas las imágenes
carpeta = './ISIC_MSKCC/Dataset/'

# Obtenemos una lista de todos los archivos en la carpeta
archivos_totales = os.listdir(carpeta)

In [ ]:
len(archivos_totales)

In [ ]:
import shutil 

# Si el archivo no es imagen dermatoscopica y existe lo eliminamos de la carpeta de imágenes
for archivo in archivos_totales:
    if archivo not in archivos_dermoscopic:
        ruta_completa = os.path.join(carpeta, archivo)
        if os.path.isfile(ruta_completa):
            os.remove(ruta_completa)
    else:
        # Obtenemos la fila específica del dataset para el archivo actual
        nombre_sin_extension, extension = os.path.splitext(archivo)
        fila_especifica = data.loc[data['id'] == nombre_sin_extension]

        # Obtenemos la etiqueta de la fila
        etiqueta = fila_especifica['benign_malignant'].values[0]  # Asumiendo que 'benign_malignant' es el nombre de la columna
        
        # Movemos el archivo según la etiqueta
        if etiqueta == 'benign':
            shutil.copy(os.path.join(carpeta, archivo), "./ISIC_MSKCC/Dataset/Benigno/")
        elif etiqueta == 'malignant':
            shutil.copy(os.path.join(carpeta, archivo), "./ISIC_MSKCC/Dataset/Maligno/")

In [ ]:
data['benign_malignant'] = data['benign_malignant'].replace(benign_malignant_mapping)

In [ ]:
# Añadimos los factores climatoloógicos obtenidos anteriormente al conjunto

# Temperatura
data['max_temp'] = 35.6
data['min_tmp'] = -10.6
data['mean_temp'] = 14.107562061212317
data['median_temp'] = 13.3
data['std_temp'] = 9.280232012833446

# Dew Point
data['max_dew_point'] = 25.0
data['min_dew_point'] = -21.1
data['mean_dew_point'] = 7.030652044234388
data['median_dew_point'] = 7.2
data['std_dew_point'] = 10.173613161292261

# Wind Speed
data['max_wind_speed'] = 79.64
data['min_wind_speed'] = 0
data['mean_wind_speed'] = 15.061415124661371
data['median_wind_speed'] = 12.96
data['std_wind_speed'] = 9.084550968803933

# Wind Direction
data['max_wind_direction'] = 360.0
data['min_wind_direction'] = 10.0
data['mean_wind_direction'] = 197.2095875709102
data['median_wind_direction'] = 220.0
data['std_wind_direction'] = 102.91723691970225

# Visibility
data['max_visibility'] = 16.09
data['min_visibility'] = 0.0
data['mean_visibility'] = 14.458736634390345
data['median_visibility'] = 16.09
data['std_visibility'] = 3.84142673414594

In [ ]:
# Leemos el nuevo dataset de calculos solares
data_calcs = pd.read_csv('./ISIC_MSKCC/MSKCC_solar_calcs.csv')
data_calcs

In [ ]:
# Transformamos el nuevo dataset para obtener una estructura entendible y facilmente iterable 
variables = [
    "ALLSKY_KT",
    "CLOUD_AMT",
    "TOA_SW_DWN",
    "ALLSKY_SFC_UVA",
    "ALLSKY_SFC_UVB",
    "ALLSKY_SRF_ALB",
    "ALLSKY_SFC_SW_DNI",
    "ALLSKY_SFC_SW_DWN",
    "ALLSKY_SFC_PAR_TOT",
    "ALLSKY_SFC_SW_DIFF"
]

columns = [
    "max",
    "min",
    "mean",
    "median",
    "std",
]

data_calcs.index = variables
data_calcs.columns = columns

data_calcs

In [ ]:
# Introducimos los nuevos datos al dataset original
for index, row in data_calcs.iterrows():
    nombre = row.name  
    for columna in data_calcs.columns:
        estadistica = columna  
        nuevo_nombre_columna = f"{nombre}_{estadistica}" 
        data[nuevo_nombre_columna] = row[columna]

In [ ]:
data

In [ ]:
# Guardamos nuestro dataset procesado para el entrenamiento
data.to_csv('./ISIC_MSKCC/final_metadata_MSKCC.csv', index=False)